In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
file1 = pd.read_csv("output_data/cities.csv")
file1.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,saldanha,0,PT,1570626065,39,41.42,-6.55,67.73,2.57
1,1,mys shmidta,100,RU,1570626065,80,62.06,48.62,35.15,7.90
2,2,katsuura,20,JP,1570626065,63,33.93,134.50,62.60,5.82
3,3,alugan,100,PH,1570626066,90,12.22,125.48,77.63,3.00
4,4,ushuaia,40,AR,1570626066,87,-54.81,-68.31,37.40,28.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [4]:
# Heatmap of humidity
fig = gmaps.figure()
locations = file1[["Lat", "Long"]].astype(float)
Humidity =file1["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
weatherdata = file1.dropna()
idealcity = weatherdata[weatherdata['Humidity']>70]
idealcity = idealcity[weatherdata['Humidity']<80]
idealcity = idealcity[weatherdata['Wind Speed']< 10]
idealcity = idealcity[weatherdata['Cloudiness'] == 0]
idealcity.head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
179,179,miyazu,0,JP,1570626116,74,35.58,135.20,51.17,1.05
243,243,vostok,0,RU,1570626132,77,46.45,135.83,41.63,6.24
246,246,talnakh,0,RU,1570626133,73,69.49,88.39,19.40,6.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel = []
idealcity['hotel'] = idealcity
latlong = idealcity[["Lat","Long"]]
idealcity

,Unnamed: 0,City,Hotel,Age,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,hotel
179,179,miyazu,True,True,0,JP,1570626116,74,35.58,135.20,51.17,1.05,179
243,243,vostok,True,True,0,RU,1570626132,77,46.45,135.83,41.63,6.24,243
246,246,talnakh,True,True,0,RU,1570626133,73,69.49,88.39,19.40,6.71,246


In [38]:
# Set parameters to search for a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location":"35.58,135.20",
    "radius":5000,
    "type": "lodging",
    "key": g_key,
}

# Iterate through 
response = requests.get(base_url, params=params).json()
results = response['results'][0]["name"]
results

'Shinpuro'

In [41]:
# Set parameters to search for a hotel
params = {
    "radius":5000,
    "type": "lodging",
    "key": g_key,
}
# Iterate through 
for index, row in idealcity.iterrows():
    lat = row['Lat']
    lng = row['Long']
    
    params['location']= f'{lat},{lng}'
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        results=response['results'][0]['name']
        idealcity.loc[index, 'Hotel Name'] = results
    
    except (KeyError, IndexError):
        idealcity.loc[index,'Hotel Name']='No Hotel'
        print("Missing field/result... skipping.")
        
idealcity.head()

Missing field/result... skipping.


,Unnamed: 0,City,Hotel,Age,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,hotel,Hotel Name
179,179,miyazu,True,True,0,JP,1570626116,74,35.58,135.20,51.17,1.05,179,Shinpuro
243,243,vostok,True,True,0,RU,1570626132,77,46.45,135.83,41.63,6.24,243,No Hotel
246,246,talnakh,True,True,0,RU,1570626133,73,69.49,88.39,19.40,6.71,246,Talnakh Hotel


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in idealcity.iterrows()]
locations = idealcity[["Lat", "Long"]]

In [46]:
# Add marker layer ontop of heat map

hotelspots = idealcity[["Lat", "Long"]]
# Display figure
gmaps.configure(api_key=g_key)


hotel_layer = gmaps.marker_layer(
    hotelspots,
    info_box_content=hotel_info
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))